In [0]:
"""
Descrição do projeto Gold

1°:Selecionar somentes colunas que o cliente/setor pediu 
2°:Criar uma nova coluna que faz a soma de todas as lesoes 
3°:Renomear colunas para ficar mais intuitivas para o usuario final 
4°:Excluir dados que estados tenham a classificação [Indeterminado, Sem Registro, Exterior]
5°:inserir coluna com nome de atualização para usuario ver quando os dados foram atualizados 
6°:Salvar na camada Gold particionada por UF > 'Estado'

"""

In [0]:
df= spark.read.parquet('dbfs:/mnt/Anac/Silver/anac_silver.parquet/')
display(df)

###### 1. Selecionar somente colunas que o cliente/setor pediu

In [0]:
colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem', 'CLS', 'Categoria_da_Aeronave', 'Classificacao_da_Ocorrência', 'Danos_a_Aeronave', 'Data_da_Ocorrencia', 'Descricao_do_Tipo', 'Fase_da_Operacao', 'Historico', 'Hora_da_Ocorrência', 'ICAO', 'Ilesos_Passageiros', 'Ilesos_Tripulantes', 'Latitude', 'Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes','UF']

df = df.select(colunas)

###### 2. Criar uma nova coluna que faz a soma de todas as lesoes


In [0]:
colunas_somar = ['Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.withColumn('Total_Lesoes', sum(df[x] for x in colunas_somar))

###### 3. Renomar Colunas

In [0]:
df = df\
  .withColumnRenamed('Aerodromo_de_Destino', 'Aeroporto_Destino')\
  .withColumnRenamed('Aerodromo_de_Origem', 'Aeroporto_Origem')\
  .withColumnRenamed('Categoria_da_Aeronave', 'Categoria_Aeronave')\
  .withColumnRenamed('Classificacao_da_Ocorrência', 'Classificacao_Ocorrencia')\
  .withColumnRenamed('Descricao_do_Tipo', 'Descricao_Acidente')

###### 4. Excluir dados que tenham classificação  ['Inderteminado','Sem_Registro','Exterior]

In [0]:
classificacao_excluir = ['Inderteminado','Sem_Registro','Exterior']
df = df.filter(~df['UF'].isin(classificacao_excluir))
display(df)

###### 5. Inserir coluna com nome de Atualização para usuario ver quando os dados foram atualizados

In [0]:
from pyspark.sql.functions import *

df = df.withColumn('Atualizacao',date_format(from_utc_timestamp(current_timestamp(),'America/Sao_Paulo'),'dd/MM -- mm:ss'))


In [0]:
display(df)

###### 6. Salvar na camada Gold particionada por UF

In [0]:
df.write.mode('overwrite').format('parquet').partitionBy('UF').save('dbfs:/mnt/Anac/Gold/anac_gold.parquet')